In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import time
from tqdm.auto import tqdm
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
squid_game = 'https://www.metacritic.com/tv/squid-game'
hellbound = 'https://www.metacritic.com/tv/hellbound'
k_zombie = 'https://www.metacritic.com/tv/all-of-us-are-dead'

dramas = [squid_game, hellbound, k_zombie]

In [4]:
chrome_driver_path = '/opt/homebrew/Caskroom/chromedriver/99.0.4844.51/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument("headless")

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s, options=options)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/suchan/.wdm/drivers/chromedriver/mac64_m1/99.0.4844.51/chromedriver] found in cache


In [5]:
def scroll_down(second):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(second)
    new = driver.execute_script("return document.documentElement.scrollHeight")
    
    return new

In [6]:
metacritic_crtic_dic = {'title' : [], 'metascore' : [], 'userscore' : [], 'critic_review' : []}

In [7]:
for url in dramas :
    driver.get(url)
    time.sleep(1)
####################################################################################################################################################################################################################################################################################################################################################################################    
    try:
        title = driver.find_element_by_css_selector('#main_content > div.movie.product.summary > div.content_under_header > div > table > tbody > tr > td.maskedcenter > div > table > tbody > tr > td.gu5.maskedbg > div > div > div.pad_btm1 > div > h1')
        metascore = driver.find_element_by_css_selector('#main_content > div.movie.product.summary > div.content_under_header > div > table > tbody > tr > td.maskedcenter > div > table > tbody > tr > td.gu5.maskedbg > div > div > div.ms_wrapper > table > tbody > tr > td.summary_right > a > span')
        userscore = driver.find_element_by_css_selector('#main_content > div.movie.product.summary > div.content_under_header > div > table > tbody > tr > td.maskedcenter > div > table > tbody > tr > td.gu5.maskedbg > div > div > div.us_wrapper > div > table > tbody > tr > td.summary_right.pad_btm1 > a > span')
    except:
        title = driver.find_element_by_css_selector('#main_content > div.movie.product.summary > div.content_under_header.no_trailer > div > div > div.fxdcol.gu12.pad_btm1 > div > h1''')
        metascore = driver.find_element_by_css_selector('#main_content > div.movie.product.summary > div.content_under_header.no_trailer > div > div > div:nth-child(2) > div > table > tbody > tr > td.summary_right > a > span')
        userscore = driver.find_element_by_css_selector('#main_content > div.movie.product.summary > div.content_under_header.no_trailer > div > div > div:nth-child(4) > div.us_wrapper > div > table > tbody > tr > td.summary_right.pad_btm1 > a > span')

    title = title.text
    metascore = metascore.text
    userscore = userscore.text
    
####################################################################################################################################################################################################################################################################################################################################################################################
    critic_count = 0
    for i in range(3):
        sentiment_num = driver.find_element_by_xpath(f'/html/body/div[2]/div/div/div[1]/div[2]/div[3]/div[1]/div[1]/div[1]/div[2]/div[2]/span[{i+1}]/div[2]/div[2]/div[2]').text
        critic_count += int(sentiment_num)
    
    try:
        driver.find_element_by_css_selector('#nav_to_metascore > div:nth-child(1) > div:nth-child(5) > a').click()
        time.sleep(1)
        for i in range(critic_count+1):
            try:
                driver.find_element_by_xpath(f'/html/body/div[2]/div/div/div[1]/div[3]/div/div[1]/div[5]/div/div[{i+1}]/div[2]/div[2]/a[1]').click()
                driver.switch_to.window(driver.window_handles[-1]) #새 창으로 옮기기
                driver.implicitly_wait(5)
                last_height = driver.execute_script("return document.documentElement.scrollHeight")
                while True:
                    new_height = scroll_down(2)
                    if new_height == last_height:
                        new_height = scroll_down(2)
                        if new_height == last_height:
                            break
                    last_height = new_height
                driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
                p_elements = driver.find_elements_by_tag_name("p")
                for p in p_elements:
                    p_text = p.text
                    metacritic_crtic_dic['title'].append(title)
                    metacritic_crtic_dic['metascore'].append(metascore)
                    metacritic_crtic_dic['userscore'].append(userscore)
                    metacritic_crtic_dic['critic_review'].append(p_text)
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(2)
            except:
                continue
    except:
        for i in range(critic_count):
            driver.find_element_by_xpath(f'/html/body/div[2]/div/div/div[1]/div[2]/div[3]/div[1]/div[1]/div[1]/div[4]/div/div[{i+1}]/div[2]/div/a[1]').click()
            driver.switch_to.window(driver.window_handles[-1]) #새 창으로 옮기기
            driver.implicitly_wait(5)
            last_height = driver.execute_script("return document.documentElement.scrollHeight")
            while True:
                new_height = scroll_down(2)
                if new_height == last_height:
                    new_height = scroll_down(2)
                    if new_height == last_height:
                        break
                last_height = new_height
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            p_elements = driver.find_elements_by_tag_name("p")
            for p in p_elements:
                p_text = p.text
                metacritic_crtic_dic['title'].append(title)
                metacritic_crtic_dic['metascore'].append(metascore)
                metacritic_crtic_dic['userscore'].append(userscore)
                metacritic_crtic_dic['critic_review'].append(p_text)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(2)
############################################################################################################################################################################################################################
    print("done")

done
done
done


In [8]:
df_metacritic_critic = pd.DataFrame(metacritic_crtic_dic)
df_metacritic_critic

,title,metascore,userscore,critic_review
0,Squid Game,69,7.1,"“Okay,” says the incredulous and increasingly ..."
1,Squid Game,69,7.1,READ MORE: Fall 2021 TV Preview: 38 Series To ...
2,Squid Game,69,7.1,"Gi-hun (Lee Jung-jae), the officer’s audience,..."
3,Squid Game,69,7.1,Netflix’s latest (and completely unexpected) h...
4,Squid Game,69,7.1,READ MORE: ‘Introduction’ Is Hong Sang-Soo Str...
...,...,...,...,...
854,All of Us Are Dead,67,7.3,What this particular zombie series can instead...
855,All of Us Are Dead,67,7.3,Most of them might be dead. But while some of ...
856,All of Us Are Dead,67,7.3,“All of Us Are Dead” is now available to strea...
857,All of Us Are Dead,67,7.3,A Variety and iHeartRadio Podcast


In [9]:
df_metacritic_critic.to_csv("metacritic_critic_reviews")